
# Bell Test with Qiskit


In [23]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np


## Function to Prepare the Bell State


In [ ]:
def bell_circuit():
    # This is the Phi+ Bell state circuit (one out of four Bell states)
    qc = QuantumCircuit(2)
    qc.h(0)        # Hadamard on qubit 0
    qc.cx(0, 1)    # CNOT with qubit 0 as control and qubit 1 as target
    return qc

## Circuit Visualization

Let's visualize the Bell state preparation and the four different measurement circuits used in the CHSH test.

In [25]:
# 1. Bell State Preparation Circuit
print("1. Bell State Preparation |Phi+⟩ = (|00⟩ + |11⟩)/sqrt(2)")
print("="*50)
bell_qc = bell_circuit()
print(bell_qc.draw())

print("\n\n2. CHSH Measurement Circuits")
print("="*50)

# Setting 1: Alice Z (0°), Bob at 22.5°
print("\nSetting 1: Alice Z-basis, Bob +22.5°")
qc1 = bell_circuit()
qc1.ry(np.pi/8, 1)  # Bob: 22.5° rotation
qc1.measure_all()
print(qc1.draw())

# Setting 2: Alice Z (0°), Bob at -22.5°
print("\nSetting 2: Alice Z-basis, Bob -22.5°")
qc2 = bell_circuit()
qc2.ry(-np.pi/8, 1)  # Bob: -22.5° rotation
qc2.measure_all()
print(qc2.draw())

# Setting 3: Alice X (45°), Bob at 22.5°
print("\nSetting 3: Alice X-basis, Bob +22.5°")
qc3 = bell_circuit()
qc3.h(0)  # Alice: X measurement
qc3.ry(np.pi/8, 1)  # Bob: 22.5° rotation
qc3.measure_all()
print(qc3.draw())

# Setting 4: Alice X (45°), Bob at -22.5°
print("\nSetting 4: Alice X-basis, Bob -22.5°")
qc4 = bell_circuit()
qc4.h(0)  # Alice: X measurement
qc4.ry(-np.pi/8, 1)  # Bob: -22.5° rotation
qc4.measure_all()
print(qc4.draw())

1. Bell State Preparation |Phi+⟩ = (|00⟩ + |11⟩)/sqrt(2)
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


2. CHSH Measurement Circuits

Setting 1: Alice Z-basis, Bob +22.5°
        ┌───┐                 ░ ┌─┐   
   q_0: ┤ H ├──■──────────────░─┤M├───
        └───┘┌─┴─┐┌─────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/8) ├─░──╫─┤M├
             └───┘└─────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════╩══╩═
                                 0  1 

Setting 2: Alice Z-basis, Bob -22.5°
        ┌───┐                  ░ ┌─┐   
   q_0: ┤ H ├──■───────────────░─┤M├───
        └───┘┌─┴─┐┌──────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/8) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 

Setting 3: Alice X-basis, Bob +22.5°
        ┌───┐        ┌───┐    ░ ┌─┐   
   q_0: ┤ H ├──■─────┤ H ├────░─┤M├───
        └───┘┌─┴─┐┌──┴───┴──┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/8) ├─░──╫─┤M├
         

## Understanding the Measurement Angles

### Why 22.5° for Bob?

**Alice's measurements:**
- Z-basis: 0° direction
- X-basis: 90° direction (after H gate)

**Bob's measurements:**
- +22.5° rotation 
- -22.5° rotation

22.5° is exactly halfway between 0° and 45°. This creates optimal quantum correlations:

## What Happens in Each Measurement

### The Four CHSH Settings:

1. **Setting 1**: Alice Z-basis (0°) + Bob +22.5°
   - High correlation expected
   
2. **Setting 2**: Alice Z-basis (0°) + Bob -22.5° 
   - High correlation expected
   
3. **Setting 3**: Alice X-basis (90°) + Bob +22.5°
   - Medium correlation expected
   
4. **Setting 4**: Alice X-basis (90°) + Bob -22.5°
   - Anti-correlation expected

### The Inequality:
**CHSH = E1 + E2 + E3 - E4**

Classical physics: CHSH <= 2  
Quantum mechanics: CHSH can reach 2*sqrt(2) = 2.83

The negative sign on E4 turns the anti-correlation into a positive contribution to the CHSH violation.

## CHSH Test Implementation and Results

In [26]:
def expectation(counts):
    total = sum(counts.values())
    exp = 0
    for outcome, count in counts.items():
        # outcome:'10' means qubit 1=1, qubit 0=0
        a = 1 if outcome[1] == '0' else -1  # Alice
        b = 1 if outcome[0] == '0' else -1  # Bob
        exp += a * b * count
    return exp / total

def run_chsh_corrected():
    """Corrected CHSH test with proper measurement angles"""
    backend = AerSimulator()
    shots = 10000
    results = []
    
    # Setting 1: Alice Z (0°), Bob at 22.5°
    qc = bell_circuit()
    qc.ry(np.pi/8, 1)  # Bob: 22.5° rotation
    qc.measure_all()
    job = backend.run(qc, shots=shots)
    results.append(job.result().get_counts())
    
    # Setting 2: Alice Z (0°), Bob at -22.5°
    qc = bell_circuit()
    qc.ry(-np.pi/8, 1)  # Bob: -22.5° rotation
    qc.measure_all()
    job = backend.run(qc, shots=shots)
    results.append(job.result().get_counts())
    
    # Setting 3: Alice X (45°), Bob at 22.5°
    qc = bell_circuit()
    qc.h(0)  # Alice: X measurement
    qc.ry(np.pi/8, 1)  # Bob: 22.5° rotation
    qc.measure_all() 
    job = backend.run(qc, shots=shots)
    results.append(job.result().get_counts())
    
    # Setting 4: Alice X (45°), Bob at -22.5°
    qc = bell_circuit()
    qc.h(0)  # Alice: X measurement
    qc.ry(-np.pi/8, 1)  # Bob: -22.5° rotation
    qc.measure_all()
    job = backend.run(qc, shots=shots)
    results.append(job.result().get_counts())
    
    return results

def compute_chsh():
    results = run_chsh_corrected()
    E = [expectation(c) for c in results]
    S = E[0] + E[1] + E[2] - E[3]
    print(f"Expectation values: {[f'{e:.3f}' for e in E]}")
    print(f"CHSH S = {S:.3f}")
    print(f"Theoretical maximum: 2*sqrt(2) ≈ {2*np.sqrt(2):.3f}")
    if abs(S) > 2:
        print("Bell inequality violated! Quantum entanglement observed.")
    else:
        print("No violation. Results compatible with classical physics.")

compute_chsh()

Expectation values: ['0.928', '0.929', '-0.403', '0.358']
CHSH S = 1.095
Theoretical maximum: 2*sqrt(2) ≈ 2.828
No violation. Results compatible with classical physics.
